In [40]:
import torch
import torch.nn.functional as F

criterion = torch.nn.BCEWithLogitsLoss()
pred = torch.rand([256, 10])
num_classes = 10
target = torch.randint(0, 9, (256,))
target = F.one_hot(target, num_classes).float()
criterion(pred, target)

tensor(0.9365)

In [47]:
criterion._get_name() == "BCEWithLogitsLoss"

True

In [6]:
import os

for d in os.listdir('./outputs/dataset=MNIST/FSW/EER'):
    print(d.split("_"))
    break

['seed=10', 'model=MLP', 'epoch=1', 'lr=0.001', 'tau=1.0', 'alpha=0.0005', 'lmbd=1.0', 'lmbdold=0.0']


In [12]:
import numpy as np

def is_number(value):
    flag=True
    try:
        num=float(value)
        flag = num == num
    except ValueError:
        flag=False
    return flag

In [17]:
runs = list()
for d in os.listdir('./outputs/dataset=MNIST/FSW/EER'):
    info_dict = dict()
    info_dict['path'] = d
    for elem in d.split("_"):
        k, v = elem.split("=")
        info_dict[k] = int(v) if v.isdigit() else float(v) if is_number(v) else v
        info_dict['path'] = d
    runs.append(info_dict)


In [16]:
self.class_idx

False

In [15]:
import torch
import numpy as np

class_idx = [8, 2, 4, 1, 5, 6, 7, 0, 3, 9]


In [18]:
from cl_gym.benchmarks.base import SplitDataset

class SplitDataset1(SplitDataset):
    def __init__(self, task_id, num_classes_per_split, dataset, class_idx = None):
        self.task_id = task_id
        self.num_classes_per_split = num_classes_per_split
        self.dataset = dataset
        if isinstance(dataset.targets, np.ndarray):
            original_target = dataset.targets
        elif isinstance(dataset.targets, list):
            original_target = np.asarray(dataset.targets)
        # for MNIST-like datasets where targets are tensors
        else:
            original_target = dataset.targets.clone().detach().numpy()
        self.original_target = original_target
        self.class_idx = np.unique(original_target) if class_idx is None else class_idx
        self.build_split(task_id)
        self.sample_weight = torch.ones(self.__len__()) #ADDED - for dtype agreement
    
    def update_weight(self, sample_weight):
        self.sample_weight = sample_weight

    def build_split(self, task_id):
        target_classes = list()
        if isinstance(task_id, int):
            task_id = [task_id]
        for task in task_id:
            start_class = (task-1) * self.num_classes_per_split
            end_class = min(task * self.num_classes_per_split, len(self.class_idx))
            target_classes += self.class_idx[start_class:end_class]
        indices = np.zeros_like(self.original_target)
        for c in target_classes:
            indices = np.logical_or(indices, self.original_target == c)
        self.true_index = np.where(indices)[0] 
        self.targets = self.original_target[self.true_index]

    def __getitem__(self, index: int):
        idx = self.true_index[index]
        img, target, *_ = self.dataset[idx]
        if isinstance(self.task_id, int):
            task_id = self.task_id
        else:
            task_id = self.class_idx.index(target) // self.num_classes_per_split + 1
        sample_weight = self.sample_weight[index]
        return img, target, task_id, index, sample_weight
    
    def __len__(self):
        return len(self.true_index)



In [26]:
from cl_gym.benchmarks.utils import DEFAULT_DATASET_DIR
import torchvision

trains = dict()
mnist_train = torchvision.datasets.MNIST(DEFAULT_DATASET_DIR, train=True, download=True)
trains[2] = SplitDataset1([1, 2], 2, mnist_train, class_idx=class_idx)

In [37]:
trains[2][8]

(<PIL.Image.Image image mode=L size=28x28>, 8, 1, 8, tensor(1.))

In [38]:
a = list()
b = np.array([1, 2])
a.extend(b)